In [18]:
import re

In [19]:
text = "Paul Newman was an American Actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

pattern = r"Paul [A-Z]\w+"

matches = re.finditer(pattern, text)

for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [20]:
import spacy
from spacy.tokens import Span

In [21]:
nlp = spacy.blank("en")
doc = nlp(text)

original_ents = list(doc.ents)

mwt_ents = []

for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start,end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="PERSON")
    original_ents.append(per_ent)

doc.ents = original_ents

print(doc.ents)

for ent in doc.ents:
    print(ent.text, ent.label_)


(Paul Newman, Paul Hollywood)
Paul Newman PERSON
Paul Hollywood PERSON


In [22]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [23]:
from spacy.language import Language

@Language.component("paul_ner")

def paul_ner(doc):
    original_ents = list(doc.ents)
    pattern = r"Paul [A-Z]\w+"
    mwt_ents = []

    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start,end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="PERSON")
        original_ents.append(per_ent)

    doc.ents = original_ents
    
    return doc
    

In [ ]:
nlp.remove_pipe("paul_ner")

In [ ]:
nlp.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [ ]:
nlp.analyze_pipes()

{'summary': {'paul_ner': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'paul_ner': []},
 'attrs': {}}

In [ ]:
text = "Paul Newman was an American Actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
Paul Waswas PERSON
Paul Hollywood PERSON


In [30]:
from spacy.language import Language
from spacy.util import filter_spans

@Language.component("cinema_ner")

def cinema_ner(doc):
    pattern = r"Hollywood"
    original_ents = list(doc.ents)
    mwt_ents = []

    for match in re.finditer(pattern,doc.text):
        start,end = match.span()
        span = doc.char_span(start,end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    
    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="CINEMA")
        original_ents.append(per_ent)
    
    filtered = filter_spans(original_ents)

    doc.ents = filtered

    return doc


In [31]:
nlp2 = spacy.load("en_core_web_sm")
nlp2.add_pipe("cinema_ner")

<function __main__.cinema_ner(doc)>

In [32]:
doc2 = nlp2(text)
for ent in doc2.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
Paul PERSON
